LIGGGHTS INP Maker

In [3]:
import re
from pathlib import Path
import pandas as pd
import sympy  

# --------- Paths (change if needed) ---------
EXCEL_PATH    = Path("PSD.xlsx")             # Update this with the actual file path
TEMPLATE_PATH = Path("sample.txt")
OUTPUT_PATH   = Path("in.generate")         # Update this with the actual file path
# --------------------------------------------

# Column names in your Excel 
RADIUS_COLUMN = "radius"
WEIGHT_COLUMNS_TRY = ["Probability Density"]    # Change it with your parameter ( e.g. volume fraction)

# Formatting knobs
FIRST_LINE_PAIRS = 3     # pts1..pts3 on first line
MAX_WIDTH_FLOOR  = 130    # minimum wrap width if pts lines are short
WEIGHT_SIG       = 8    # significant digits for printed weights (high precision)

# Defaults if not found in the template’s existing pts line
DEFAULT_ATOM_TYPE = 1
DEFAULT_DENSITY   = 30000


# ---------- helpers ----------
def next_prime_after(n: int) -> int:
    return int(sympy.nextprime(n))


def read_template(path: Path):
    with path.open("r", encoding="utf-8") as f:
        return f.readlines()


def largest_seed_in_file(lines):
    seeds = []
    rx = re.compile(r"\b([1-9]\d{6,})\b")
    for line in lines:
        for m in rx.findall(line):
            try:
                seeds.append(int(m))
            except:
                pass
    return max(seeds) if seeds else 32452843  


def infer_atom_density(lines):
    atom_type, density = DEFAULT_ATOM_TYPE, DEFAULT_DENSITY
    rx = re.compile(
        r"^\s*fix\s+pts\d+\s+all\s+particletemplate/sphere\s+\S+\s+atom_type\s+(\d+).*?"
        r"density\s+constant\s+([0-9.eE+-]+)"
    )
    for line in lines:
        m = rx.match(line)
        if m:
            atom_type = int(m.group(1))
            try:
                density = float(m.group(2))
            except ValueError:
                pass
            break
    return atom_type, density


def load_radii_weights(xlsx: Path):
    df = pd.read_excel(xlsx)
    if RADIUS_COLUMN not in df.columns:
        raise ValueError(f"Radius column '{RADIUS_COLUMN}' not found in {xlsx.name}")

    weight_col = None
    for cand in WEIGHT_COLUMNS_TRY:
        if cand in df.columns:
            weight_col = cand
            break
    if weight_col is None:
        raise ValueError(f"No weight column found (tried: {WEIGHT_COLUMNS_TRY})")

    radii   = df[RADIUS_COLUMN].astype(float)
    weights = df[weight_col].astype(float)

    mask = weights.fillna(0.0) > 0
    return radii[mask].to_numpy(), weights[mask].to_numpy()


def build_pts_lines(radii, seeds, atom_type, density):
    pt_lines, pt_names = [], []
    for i, (r, s) in enumerate(zip(radii, seeds), start=1):
        name = f"pts{i}"
        pt_names.append(name)
        pt_lines.append(
            f"fix\t\t{name} all particletemplate/sphere {s} atom_type {atom_type} "
            f"density constant {density:g} radius constant {r:.12g}\n"
        )
    return pt_lines, pt_names


def compute_indent_under_pdd1(header: str):
    prefix_to_pdd1 = header[:header.index("pdd1")]
    indent_width = len(prefix_to_pdd1.expandtabs(4))
    return " " * indent_width


def build_pdd_block(pt_names, weights, pdd_seed, pt_lines_before):
    fmt_w = lambda x: f"{float(x):.{WEIGHT_SIG}g}"
    pairs = [f"{name} {fmt_w(w)}" for name, w in zip(pt_names, weights)]

    header = f"fix\t\tpdd1 all particledistribution/discrete {pdd_seed} {len(pt_names)} "
    indent = compute_indent_under_pdd1(header)

    # wrap width from the longest preceding pts line (visual width)
    pts_width = max((len(L.expandtabs(4).rstrip("\n")) for L in pt_lines_before[-20:]), default=0)
    max_width = max(MAX_WIDTH_FLOOR, pts_width)

    lines = []
    # first line: only FIRST_LINE_PAIRS (so pts4 starts next line)
    lines.append(header + " ".join(pairs[:FIRST_LINE_PAIRS]))

    # subsequent lines: fill to width, all starting at indent
    current = indent
    for pair in pairs[FIRST_LINE_PAIRS:]:
        candidate = (current if current == indent else current + " ") + pair
        if len(candidate.expandtabs(4)) > max_width and current != indent:
            lines.append(current)
            current = indent + pair
        else:
            current = candidate
    if current.strip():
        lines.append(current)

    # ampersand on all but last line
    for i in range(len(lines) - 1):
        lines[i] = lines[i] + " &"
    return "\n".join(lines) + "\n"


def replace_pts_and_pdd(template_lines, pt_lines, pdd_block):
    new_lines, i, replaced = [], 0, False
    rx_pts = re.compile(r"^\s*fix\s+pts\d+\s+all\s+particletemplate/sphere")
    rx_pdd = re.compile(r"^\s*fix\s+pdd1\s+all\s+particledistribution/discrete")

    while i < len(template_lines):
        line = template_lines[i]
        if rx_pts.match(line):
            while i < len(template_lines) and rx_pts.match(template_lines[i]):
                i += 1
            while i < len(template_lines) and rx_pdd.match(template_lines[i]):
                i += 1
            new_lines.extend(pt_lines)
            new_lines.append(pdd_block)
            replaced = True
            continue
        elif rx_pdd.match(line):
            new_lines.extend(pt_lines)
            new_lines.append(pdd_block)
            replaced = True
            i += 1
            continue
        else:
            new_lines.append(line)
            i += 1

    if not replaced:
        new_lines.extend(["\n# Autogenerated particle templates & distribution\n"] + pt_lines + [pdd_block])
    return new_lines
# ---------------------------


def main():
    tmpl_lines = read_template(TEMPLATE_PATH)
    radii, weights = load_radii_weights(EXCEL_PATH)
    atom_type, density = infer_atom_density(tmpl_lines)

    # ---- SEEDS (fixed) ----
    base = largest_seed_in_file(tmpl_lines)
    pdd_seed = next_prime_after(base)
    # IMPORTANT FIX: start pts* seeds AFTER the pdd1 seed
    cur = pdd_seed
    pt_seeds = []
    for _ in range(len(radii)):
        cur = next_prime_after(cur)
        pt_seeds.append(cur)
    # -----------------------

    pt_lines, pt_names = build_pts_lines(radii, pt_seeds, atom_type, density)
    pdd_block = build_pdd_block(pt_names, weights, pdd_seed, pt_lines)

    out_lines = replace_pts_and_pdd(tmpl_lines, pt_lines, pdd_block)
    with OUTPUT_PATH.open("w", encoding="utf-8") as f:
        f.writelines(out_lines)

    print(f"{OUTPUT_PATH} is ready to use.")
    print(f"Entries: {len(pt_names)}")


if __name__ == "__main__":
    main()


in.generate is ready to use.
Entries: 35
